In [6]:
pip install fastapi


In [7]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.6 MB/s eta 0:00:00


In [8]:
pip install python-jose

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: pyasn1
    Found existing installation: pyasn1 0.6.1
    Uninstalling pyasn1-0.6.1:
      Successfully uninstalled pyasn1-0.6.1


In [10]:
pip install requests

In [13]:
from fastapi import FastAPI, Depends, WebSocket, WebSocketDisconnect
from fastapi.middleware.cors import CORSMiddleware
from pymongo import MongoClient
from jose import jwt
import requests, time, threading
from typing import List

app = FastAPI()
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_credentials=True, allow_methods=["*"], allow_headers=["*"])

mongo = MongoClient("mongodb://localhost:27017/")
db = mongo.stock_watchlist
JWT_SECRET = "supersecretkey"
STOCK_API = {"key": "YOUR_STOCK_API_KEY", "url": "https://www.alphavantage.co/query"}

@app.post("/register")
async def register(user: dict):
    if db.users.find_one({"username": user["username"]}):
        return {"error": "User exists"}
    db.users.insert_one(user)
    return {"message": "Registered successfully"}

@app.post("/login")
async def login(user: dict):
    db_user = db.users.find_one({"username": user["username"]})
    if not db_user or db_user["password"] != user["password"]:
        return {"error": "Invalid credentials"}
    return {"access_token": jwt.encode({"sub": user["username"]}, JWT_SECRET, algorithm="HS256")}

@app.post("/watchlist")
async def add_to_watchlist(stock: dict):
    db.watchlists.update_one({"username": stock["username"]}, {"$addToSet": {"stocks": stock["symbol"]}}, upsert=True)
    return {"message": "Stock added"}

@app.get("/stocks/{symbol}")
async def get_stock_price(symbol: str):
    response = requests.get(STOCK_API["url"], params={"function": "GLOBAL_QUOTE", "symbol": symbol, "apikey": STOCK_API["key"]})
    return response.json()

class WebSocketManager:
    def __init__(self):
        self.active_connections: List[WebSocket] = []

    async def connect(self, websocket: WebSocket):
        await websocket.accept()
        self.active_connections.append(websocket)

    def disconnect(self, websocket: WebSocket):
        self.active_connections.remove(websocket)

    async def broadcast(self, message: dict):
        for connection in self.active_connections:
            await connection.send_json(message)

manager = WebSocketManager()

@app.websocket("/ws/{symbol}")
async def websocket_endpoint(websocket: WebSocket, symbol: str):
    await manager.connect(websocket)
    try:
        while True:
            response = requests.get(STOCK_API["url"], params={"function": "GLOBAL_QUOTE", "symbol": symbol, "apikey": STOCK_API["key"]})
            await manager.broadcast(response.json())
            time.sleep(10)
    except WebSocketDisconnect:
        manager.disconnect(websocket)

@app.post("/set_alert")
async def set_price_alert(alert: dict):
    db.alerts.insert_one(alert)
    return {"message": "Alert set"}

def check_alerts():
    while True:
        for alert in db.alerts.find():
            response = requests.get(STOCK_API["url"], params={"function": "GLOBAL_QUOTE", "symbol": alert["symbol"], "apikey": STOCK_API["key"]})
            price = response.json().get("Global Quote", {}).get("05. price")
            if price and float(price) >= alert["price"]:
                db.alerts.delete_one({"_id": alert["_id"]})
        time.sleep(30)

threading.Thread(target=check_alerts, daemon=True).start()
